In [1]:
from pycocotools.coco import COCO
import requests

# instantiate COCO specifying the annotations json path
coco = COCO('/home/asr/projects/mobile_net_IH/data/annotations/instances_train2017.json')
# Specify a list of category names of interest
catIds = coco.getCatIds(catNms=['bus'])
# Get the corresponding image ids and images using loadImgs
imgIds = coco.getImgIds(catIds=catIds)
images = coco.loadImgs(imgIds)

for im in images:
    img_data = requests.get(im['coco_url']).content
    with open('/home/asr/projects/mobile_net_IH/data/Bus' + im['file_name'], 'wb') as handler:
        handler.write(img_data)

ModuleNotFoundError: No module named 'pycocotools'

In [ ]:
import torchvision
import torch
from torchvision import transforms
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from zipfile import ZipFile
import random
import PIL
import os
from matplotlib import pyplot as plt
from torch import nn

SEED EVERYTHING

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
torch.backends.cuda.matmul.allow_tf32 = True # for better mutrix multiplication

LOAD DATA

In [ ]:
input_dir = 'data'
batch_size = 64
rescale_size = 224
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

Convert the image to a 255 x 255 image and normalize by mean and standard deviation from ImageNet

In [ ]:
transform = transforms.Compose([
    transforms.Resize((int(rescale_size * 1.25), int(rescale_size * 1.25))),
    transforms.RandomCrop(rescale_size),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.8),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
dataset_full = torchvision.datasets.ImageFolder(root=input_dir, transform=transform)

# split full dataset
train_idx, valid_idx = train_test_split(list(range(len(dataset_full))), train_size=0.88)
dataset = {
    'train': torch.utils.data.Subset(dataset_full, train_idx),
    'valid': torch.utils.data.Subset(dataset_full, valid_idx)
}

dataset_size = {ds: len(dataset[ds]) for ds in ['train', 'valid']}
dataset_classes = np.array(dataset_full.classes)
print('classes:', dataset_classes, '\nsize', dataset_size)

dataloader = {
    'train': torch.utils.data.DataLoader(
        dataset=dataset['train'], batch_size=batch_size, shuffle=True, num_workers=2
    ),
    'valid': torch.utils.data.DataLoader(
        dataset=dataset['valid'], batch_size=batch_size, shuffle=False, num_workers=2
    ),
}

classes: ['CS120.01.413' 'CS120.07.442' 'CS150.01.427-01' 'SU160.00.404'
 'SU80.01.426' 'SU80.10.409A' 'ЗВТ86.103К-02' 'СВМ.37.060' 'СВМ.37.060А'
 'СВП-120.00.060' 'СВП120.42.020' 'СВП120.42.030' 'СК20.01.01.01.406'
 'СК20.01.01.02.402' 'СК30.01.01.02.402' 'СК30.01.01.03.403'
 'СК50.01.01.404' 'СК50.02.01.411' 'СПО250.14.190'] 
size {'train': 577, 'valid': 79}


In [ ]:
num_classes = len(dataset_classes)
num_classes

19

TRAINING LOOP

In [ ]:
from torch.cuda.amp import autocast, GradScaler
import tqdm
# from accelerate import Accelerator
scaler = GradScaler()
# accelerator = Accelerator(gradient_accumulation_steps=4, mixed_precision='fp16')

def train_epoch(
    model: torch.nn.Module,
    optimizer: torch.optim,
    criterion: torch.nn.modules.loss._Loss,
    loader: torch.utils.data.DataLoader,
    device: torch.device
):
    acc_loss = 0
    total = len(loader.dataset)
    # model.to(device)
    model.train()
    for data, target in tqdm.tqdm(loader):
      # with accelerator.accumulate(model): # для имитации большого размера батча (полезно для трансформеров)
        data = data.to(device)
        target = target.to(device)
        pred = model(data)
        # print(target)
        loss = criterion(pred, target)
        # scaler.scale(loss).backward()
        # scaler.unscale_(optimizer)
        # scaler.step(optimizer)
        loss.backward()
        # accelerator.backward(loss) # вместо loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        acc_loss += loss.item()
    return acc_loss / total


In [ ]:
from collections import namedtuple
from typing import NamedTuple, List

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
from collections import namedtuple
from typing import NamedTuple, List

EvalOut = namedtuple("EvalOut", ['loss', 'accuracy'])


def eval_epoch(
    model: torch.nn.Module,
    criterion: torch.nn.modules.loss._Loss,
    loader: torch.utils.data.DataLoader,
    device: torch.device
):
    acc_loss = 0
    accuracy = 0
    total = len(loader.dataset)
    model.eval()
    # model.to(device)
    with torch.inference_mode():
        for data, target in loader:
            data = data.to(device)
            target = target.to(device)
            pred = model(data)
            loss = criterion(pred, target)
            acc_loss += loss.item()
            accuracy += torch.sum(torch.argmax(pred, 1) == target).item()

    return EvalOut(loss = (acc_loss / total), accuracy = (accuracy / total))


class TrainOut(NamedTuple):
    train_loss: List[float]
    eval_loss: List[float]
    eval_accuracy: List[float]


def train(
    model: torch.nn.Module,
    optimizer: torch.optim,
    criterion: torch.nn.modules.loss._Loss,
    sheduler: torch.nn.Module,
    train_loader: torch.utils.data.DataLoader,
    val_loader: torch.utils.data.DataLoader,
    device: torch.device,
    epochs: int = 10
):
    train_loss = []
    eval_loss = []
    eval_accuracy = []
    model.to(device)
    for i in range(epochs):
        print(f"Epoch - {i}\n")
        if (train_loader != None):
            print("Train...\n")
            train_loss.append(train_epoch(model, optimizer, criterion, train_loader, device))
        print("Validation...\n")
        eval_out = eval_epoch(model, criterion, val_loader, device)
        eval_loss.append(eval_out.loss)
        eval_accuracy.append(eval_out.accuracy)
        print(f'Validation acc: {eval_out.accuracy}')
        sheduler.step()
        print('lr: ', get_lr(optimizer))
        if i > 1 and eval_accuracy[i] == max(eval_accuracy):
        # unwrapped_model = accelerator.unwrap_model(model)
        # accelerator.save({
          #    "model": model.state_dict(),
            # "optimizer": optimizer.optimizer.state_dict() # optimizer is an AcceleratedOptimizer object
          # }, "/content/drive/MyDrive/accel.pth")

          torch.save({
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          }, f'detail_checkpoint_{i}_mobilt.pth')

    return TrainOut(train_loss = train_loss,
                    eval_loss = eval_loss,
                    eval_accuracy = eval_accuracy), model


SHOW LOSSES

In [ ]:
def show_losses(TrainOut, epochs):
    plt.plot(epochs, TrainOut.train_loss)
    plt.plot(epochs, TrainOut.eval_loss)
    plt.show()

def show_accuracy(accuracy, epochs):
    plt.plot(epochs, accuracy)
    plt.show()

In [ ]:
def predict(model, dataloader_test):
    logits = []
    model.eval()
    with torch.no_grad():
        for inputs, _ in dataloader_test:
            inputs = inputs.to(device)
            outputs = model(inputs).cpu()
            logits.append(outputs)
    probs = torch.nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

DEFINE MODEL

In [ ]:
import torchvision.transforms as T
from torchvision.models import mobilenet_v3
mobilenet = mobilenet_v3(weights='IMAGENET1K_V2')

mobilenet = mobilenet_v3(weights='IMAGENET1K_V2')

for param in self.mobilenet.parameters():
    param.requires_grad = False

for j in range(5, 19):

    for param in self.mobilenet.features[j].parameters():
        param.requires_grad = True

for param in self.mobilenet.classifier.parameters():
    param.requires_grad = True

mobilenet.classifier[1] = torch.nn.Linear(1280, 2)


def forward(self, x: torch.Tensor) -> torch.Tensor:
    y = self.mobilenet(x)
    return y

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 39.2MB/s]


In [ ]:
model = mobilenet()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
sheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
epochs = 60
tr_tuple, model = train(model, optimizer, criterion, sheduler, dataloader['train'], dataloader['valid'], device, epochs)

Epoch - 0

Train...



  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 10/10 [03:33<00:00, 21.35s/it]

Validation...



Validation acc: 0.5316455696202531
lr:  0.00099
Epoch - 1

Train...



100%|██████████| 10/10 [01:44<00:00, 10.46s/it]

...

Validation acc: 0.8481012658227848
lr:  0.0008261686238355864
Epoch - 19

Train...



100%|██████████| 10/10 [01:41<00:00, 10.16s/it]

Validation...



Validation acc: 0.8481012658227848
lr:  0.0008179069375972306
Epoch - 20

Train...



 20%|██        | 2/10 [00:23<01:16,  9.57s/it]

In [ ]:
show_losses(tr_tuple, list(range(epochs)))
show_accuracy(tr_tuple.eval_accuracy, list(range(epochs)))